# Tugas II IF4044 Spark Job dengan Pyspark
Muhammad Gilang Ramadhan 13520137

In [ ]:
import json
import time
import pyspark.sql.functions as psfunc

## Youtube Dataframe

### Youtube Video

In [ ]:
print("Reading json file for youtube video, Please Wait...")
dataframe = spark.read.json(f"hdfs://localhost:8888/raw_json/youtube_video_*.json")
print("Done.")

# get distinct data frame
dist_dataframe = dataframe.coalesce(1).select("id",psfunc.to_date("snippet.publishedAt").alias("date")).distinct()
youtube_video_df = dist_dataframe.groupBy("date").agg(psfunc.count("id").alias("count"))


### Youtube Comment

In [ ]:
print("Reading json file for youtube comment, Please Wait...")
dataframe = spark.read.json(f"hdfs://localhost:8888/raw_json/youtube_comment_*.json")
print("Done.")

# get distinct dataframe
dist_dataframe = dataframe.coalesce(1).select("snippet.topLevelComment.id",psfunc.to_date("snippet.topLevelComment.snippet.publishedAt").alias("date")).distinct()
youtube_comment_df = dist_dataframe.groupBy("date").agg(psfunc.count("id").alias("count"))

In [ ]:
# Merge youtube video and youtube comment dataframe to mergeAll variable
df_merge = youtube_video_df.union(youtube_comment_df)
youtube_merge_df_dist = df_merge.coalesce(1)

mergeAll_df = youtube_merge_df_dist.select(psfunc.col("date"), psfunc.lit("youtube").alias("socmed"), psfunc.col("count"))

## Instagram Dataframe

### Instagram Comment

In [ ]:
print("Reading json file for instagram comment, Please Wait...")
dataframe = spark.read.json(f"hdfs://localhost:8888/raw_json/instagram_comment_*.json")
print("Done.")

# get distinct dataframe
dist_dataframe = dataframe.coalesce(1).select(psfunc.col("id"), psfunc.from_unixtime(timestamp="created_time", format="yyyy-MM-dd").alias("date"))
instagram_comment_dataframe = dist_dataframe

### Instagram Post

In [ ]:
print("Reading json file for instagram post, Please Wait...")
dataframe = spark.read.json(f"hdfs://localhost:8888/raw_json/instagram_post_*.json")
print("Done.")

# get distinct dataframe
dist_dataframe = dataframe.coalesce(1).select(psfunc.col("id"), psfunc.from_unixtime(timestamp="created_time", format="yyyy-MM-dd").alias("date"))
instagram_post_dataframe = dist_dataframe

### Instagram Status

In [ ]:
print("Reading json file for instagram status, Please Wait...")
dataframe = spark.read.json(f"hdfs://localhost:8888/raw_json/instagram_status_*.json")
print("Done.")

# get distinct dataframe
dist_dataframe = dataframe.coalesce(1).select(psfunc.col("id"), psfunc.from_unixtime(timestamp="created_time", format="yyyy-MM-dd").alias("date"))
instagram_status_dataframe = dist_dataframe

In [ ]:
# merge all instagram content dataframe and merge with mergeAll_df variable
df_merge = instagram_comment_dataframe.union(instagram_post_dataframe)
df_merge = instagram_status_dataframe.union(df_merge)
instagram_merge_df_dist = df_merge.coalesce(1)

temp_merge_df = instagram_merge_df_dist.select(psfunc.col("date"), psfunc.lit("instagram").alias("socmed"), psfunc.col("count")).union(mergeAll_df)
mergeAll_df = temp_merge_df.coalesce(1)


## Facebook Dataframe

### Facebook post

In [ ]:
print("Reading json file for facebook post, Please Wait...")
dataframe = spark.read.json(f"hdfs://localhost:8888/raw_json/facebook_post_*.json")
print("Done.")

# get distinct dataframe
dist_dataframe = dataframe.coalesce(1).select(psfunc.col("id"), psfunc.to_date("created_time").alias("date")).distinct()
facebook_post_dataframe = dist_dataframe.groupBy("date").agg(psfunc.count("id").alias("count"))

In [ ]:
# merge facebook post with mergeAll_df variable
temp_merge_df = facebook_post_dataframe.select(psfunc.col("date"), psfunc.lit("facebook").alias("socmed"), psfunc.col("count")).union(mergeAll_df)
mergeAll_df = temp_merge_df.coalesce(1)

## Twitter Dataframe

### Twitter Status

In [ ]:
print("Reading json file for facebook post, Please Wait...")
dataframe = spark.read.json(f"hdfs://localhost:8888/raw_json/twitter_status_*.json")
print("Done.")

# get distinct dataframe
dist_dataframe = dataframe.coalesce(1).select(psfunc.col("id"),psfunc.to_date(psfunc.substring("created_at", 5, 26), "MMM dd HH:mm:ss Z yyyy").alias("date")).distinct()
twitter_status_dataframe = dist_dataframe.groupBy("date").agg(psfunc.count("id").alias("count"))

In [ ]:
# merge twitter status with mergeAll_df variable
temp_merge_df = twitter_status_dataframe.select(psfunc.col("date"), psfunc.lit("twitter").alias("socmed"), psfunc.col("count")).union(mergeAll_df)
mergeAll_df = temp_merge_df.coalesce(1)

## Show all merged dataframe

In [ ]:
print("Writing all merged files, please wait.....")
timeCount = time.time()
mergeAll_df.write.csv(f"hdfs://localhost:8888/output_csv_tugas2")
timeCount = time.time() - timeCount
print("Files were writen, done.")
print("Duration: {} s". format(timeCount))